## Introduction to Path Analysis using Batfish

Path analysis is one of the most common tasks a network engineer will undertake, but one of the most complicated ones. Traditionally, path analysis is performed by using `traceroute`. In a lot of instances, this must be performed from multiple locations in the network. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-use queries.

In this notebook, we will look at how you can perform path analysis with Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-path-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

/Users/victor/pybatfish/pybatfish/client/commands.py:49: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### Find the path taken by AS3 Core routers to reach host1, the DNS Server in AS2

To do this, we will use the `traceroute` question in Batfish. The question has a lot of parameters that you can specify, but we will start with the simple case.

By specifying `dst = "host1"` BF will automatically pick the IP address for host1 as the destination IP address for the question. If `host1` had multiple interfaces and IP address, Batfish would pick one. Further in the notebook, we will cover how you can explicitly control which of the addresses it will choose.

We want the query to start from the `Loopback0` interface on `as3core1`, and we want to use the IP address of that interface as the source address. We can accomplish both of these goals by specifying the correct value as `traceStart`. Setting `traceStart` to `as3core1[Loopback0]` tells Batfish to start the traceroute at `as3core1` with the IP address of `Loopback0` as the source IP

In [3]:
# start the traceroute from the Loopback0 interface of as3core1 to host1
tracert = bfq.traceroute(startLocation="as3core1[Loopback0]", headers=HeaderConstraints(dstIps='ofLocation(host1)')).answer().frame()
print(tracert.columns)
print(len(tracert))

Index(['Flow', 'Traces'], dtype='object')
1


Let's take a look at the results of the query. 

Compared to running traceroute from an actual node in your network, Batfish returns additional information:
1. All active parallel paths between the source and destination
2. The reason why each hop in a path is taken (the specific routing entry that was matched)
3. Disposition of the packet for each path

First, lets find out how many parallel active paths are present between the source and destination

In [4]:
# number of paths between as3core1 and host1
path_count = len(tracert['Traces'][0])
print("Number of active paths between source and destination: {}".format(path_count))

Number of active paths between source and destination: 4


Now let's take a look at one of those paths in more detail. Let's start with the first path

In [5]:
# path number we want to see
num_path = 0
# print the hops for the first trace
path = tracert['Traces'][0][num_path]
display(path)

FlowTrace(disposition='DENIED_IN', hops=[FlowTraceHop(edge=Edge(node1='as3core1', node1interface='GigabitEthernet1/0', node2='as3border1', node2interface='GigabitEthernet0/0'), routes=['BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:3.0.1.1'], transformedFlow=None), FlowTraceHop(edge=Edge(node1='as3border1', node1interface='GigabitEthernet1/0', node2='as2border2', node2interface='GigabitEthernet0/0'), routes=['BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:10.23.21.2'], transformedFlow=None), FlowTraceHop(edge=Edge(node1='as2border2', node1interface='GigabitEthernet1/0', node2='as2core2', node2interface='GigabitEthernet0/0'), routes=['BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.22.2', 'BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.22.2'], transformedFlow=None), FlowTraceHop(edge=Edge(node1='as2core2', node1interface='GigabitEthernet2/0', node2='as2dist2', node2interface='GigabitEthernet0/0'), routes=['BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.23.22.3'], transformedFlow=None), FlowTraceHop(edge=Edge(node1='as2dist2', node1interface='GigabitEthernet2/0', node2='as2dept1', node2interface='GigabitEthernet1/0'), routes=['BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.34.201.4'], transformedFlow=None), FlowTraceHop(edge=Edge(node1='as2dept1', node1interface='GigabitEthernet2/0', node2='host1', node2interface='eth0'), routes=['ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:null'], transformedFlow=None)], notes='DENIED_IN{filter::INPUT}{default}')

Let's look at this table in a bit more detail. It has the following columns:
1. `edge`: This is telling you the link that the packet traverses
2. `filterIn`: This tells you the name of any input filter that is applied on the input interface of the 2nd node in the `edge`
3. `routes`: This tells you the exact route on the first node in the `edge` that caused the packet to traverse that `edge`

What we see from the above output is that the packet was delivered to `host1` interface `eth0` by the node `as2dept1` from interface `GigabitEthernet2/0`. So now let's see what happened to the packet at `host1`

In [6]:
print("Traceroute disposition was: {}".format(path.disposition))

Traceroute disposition was: DENIED_IN


The packet was `DENIED_IN`, which means that the input filter on the host denied the packet. Let's determine the name of the filter and the specific rule that matched the packet

In [7]:
print("Details about packet disposition: {}".format(path.notes))

Details about packet disposition: DENIED_IN{filter::INPUT}{default}


## Abstract path analysis (reachability)


Traceroute allows you to find the paths taken by a specified flow through the network, as computed by Batfish. This is very useful in analyzing a connectivity issue.

Batfish has an even more powerful query, `reachability`, that will allow you to explore the network in a more abstract fashion. This is very useful 
when you want to build a set of tests for the network to ensure security and reliability.

The `reachability` question allows you to find those needle in the haystack issues. If you want to find potential flows that will be `DROPPED` or `ACCEPTED` in the network, without advanced knowledge of the `header space (srcIp, dstIp, IpProtocol, srcPort, dstPort)` then `reachability` is the way to go.

Let's take the same start location, but instead of analyzing traffic to host1, let's analyze traffic for any host in the same subnet.
That subnet is connected to `as2dept1` interface `GigabitEthernet2/0`.

And instead of using the IP address of `as3core1` interface `Loopback0`, we will analyze the entire IPv4 address space

In [8]:
path = PathConstraints(startLocation="enter(as3core1)")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(as2dept1[GigabitEthernet2/0])")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)
# reach

,Flow,Traces
0,2.0.0.0:0 → 2.128.0.1:0start=as3core1 interface=GigabitEthernet0/0 vrf=None,"DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as3core1:GigabitEthernet0/0Route(s): 2 as3core1:GigabitEthernet1/0 → as3border1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:3.0.1.13 as3border1:GigabitEthernet1/0 → as2border2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:10.23.21.2"
1,2.0.0.0:0 → 2.128.0.1:0start=as3core1 interface=GigabitEthernet1/0 vrf=None,"DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as3core1:GigabitEthernet1/0Route(s): 2 as3core1:GigabitEthernet1/0 → as3border1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:3.0.1.13 as3border1:GigabitEthernet1/0 → as2border2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:10.23.21.2"
2,2.0.0.0:0 → 2.128.0.1:0start=as3core1 interface=GigabitEthernet2/0 vrf=None,"DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as3core1:GigabitEthernet2/0Route(s): 2 as3core1:GigabitEthernet1/0 → as3border1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:3.0.1.13 as3border1:GigabitEthernet1/0 → as2border2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:10.23.21.2"
3,2.0.0.0:0 → 2.128.0.1:0start=as3core1 interface=GigabitEthernet3/0 vrf=None,"DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as3core1:GigabitEthernet3/0Route(s): 2 as3core1:GigabitEthernet1/0 → as3border1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:3.0.1.13 as3border1:GigabitEthernet1/0 → as2border2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:10.23.21.2"
4,2.0.0.0:0 → 2.128.0.1:0start=as3core1 interface=Loopback0 vrf=None,"DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as3core1:Loopback0Route(s): 2 as3core1:GigabitEthernet1/0 → as3border1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:3.0.1.13 as3border1:GigabitEthernet1/0 → as2border2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/16,nhip:10.23.21.2,nhint:dynamic>_fnhip:10.23.21.2"


Now, what if we wanted to know what flows that are not generated by sources in your network, would be able to reach `host1`, we would run the following `reachability` query

In [9]:
path = PathConstraints(startLocation="as2border.*")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)

,Flow,Traces
0,0.0.0.0:0 → 2.128.0.101:0start=as2border1,"DENIED_IN{filter::INPUT}{default}1 as2border1:GigabitEthernet1/0 → as2core1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.11.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.11.22 as2core1:GigabitEthernet2/0 → as2dist1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.23.11.33 as2dist1:GigabitEthernet2/0 → as2dept1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.34.101.44 as2dept1:GigabitEthernet2/0 → host1:eth0Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullDENIED_IN{filter::INPUT}{default}1 as2border1:GigabitEthernet1/0 → as2core1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.11.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.11.22 as2core1:GigabitEthernet3/0 → as2dist2:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.23.12.33 as2dist2:GigabitEthernet2/0 → as2dept1:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.34.201.44 as2dept1:GigabitEthernet2/0 → host1:eth0Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullDENIED_IN{filter::INPUT}{default}1 as2border1:GigabitEthernet2/0 → as2core2:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.12.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.12.22 as2core2:GigabitEthernet2/0 → as2dist2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.23.22.33 as2dist2:GigabitEthernet2/0 → as2dept1:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.34.201.44 as2dept1:GigabitEthernet2/0 → host1:eth0Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullDENIED_IN{filter::INPUT}{default}1 as2border1:GigabitEthernet2/0 → as2core2:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.12.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.12.22 as2core2:GigabitEthernet3/0 → as2dist1:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.23.21.33 as2dist1:GigabitEthernet2/0 → as2dept1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.34.101.44 as2dept1:GigabitEthernet2/0 → host1:eth0Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:null"
1,0.0.0.0:0 → 2.128.0.101:0start=as2border2,"DENIED_IN{filter::INPUT}{default}1 as2border2:GigabitEthernet1/0 → as2core2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.22.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.22.22 as2core2:GigabitEthernet2/0 → as2dist2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.23.22.33 as2dist2:GigabitEthernet2/0 → as2dept1:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.34.201.44 as2dept1:GigabitEthernet2/0 → host1:eth0Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullDENIED_IN{filter::INPUT}{default}1 as2border2:GigabitEthernet1/0 → as2core2:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.22.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.22.22 as2core2:GigabitEthernet3/0 → as2dist1:GigabitEthernet1/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.23.21.33 as2dist1:GigabitEthernet2/0 → as2dept1:GigabitEthernet0/0Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.34.101.44 as2dept1:GigabitEthernet2/0 → host1:eth0Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthern

As you can see, this query identified flows entering the network at both `as2border1` and `as2border2` destined for `host1` that would be delivered.
And in all cases, it was DNS traffic that was permitted, as we would have expected based on the configuration of `host1`

### Check if *any* DNS flow originating inside AS2 destined for `host1` is dropped

In [10]:
path = PathConstraints(startLocation="as2border.*")
headers = HeaderConstraints(dstIps="ofLocation(host1)", applications=["DNS"])
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)

,Flow,Traces


### Check if *any* DNS flow originating outside of AS2 destined for `host1` is dropped.

In [11]:
path = PathConstraints(startLocation="enter(as2border.*)")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", applications=["DNS"])
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)

,Flow,Traces
0,2.0.0.0:0 → 2.128.0.101:53start=as2border1 interface=GigabitEthernet0/0 vrf=None,DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as2border1:GigabitEthernet0/0Route(s):
1,2.0.0.0:0 → 2.128.0.101:53start=as2border2 interface=GigabitEthernet0/0 vrf=None,DENIED_IN{OUTSIDE_TO_INSIDE}{deny ip 2.0.0.0 0.255.255.255 any}1 traceroute_source_node:traceroute_source_interface → as2border2:GigabitEthernet0/0Route(s):


### Find any flows from outside AS2 that would be accepted by `host1` that are not DNS or SSH

In [15]:
path = PathConstraints(startLocation="enter(as2border.*)")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", ipProtocols=["TCP", "UDP"], dstPorts="!22,!53")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="accepted").answer().frame()
display_html(reach)

,Flow,Traces


### Wrap-up

This concludes the notebook. To recap, in this notebook we covered the foundational tasks for path analysis:

1. Retrieving the list of IP addresses configured on every interface on every device in the snapshot
2. Determining the paths taken, and disposition along each path, for a flow using `traceroute`

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics like path analysis, debugging ACLs and firewall rules, validating routing policy, etc.. so stay tuned! 

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.